connect the API with function calling to provide structured output.

In [ ]:
! pip install -q openai pydantic

In [ ]:
from enum import Enum
import json
import requests
import openai
from openai import OpenAI

In [ ]:
from google.colab import userdata

openai_api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
client = OpenAI(api_key=openai_api_key)
MODEL = "gpt-4o-2024-08-06"

In [ ]:
weather_api_key = userdata.get('openweather')

In [ ]:
def get_weather(city: str):
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api_key}&units=metric'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        weather = {
            "city": city,
            "temperature": data['main']['temp'],
            "description": data['weather'][0]['description'],
            "humidity": data['main']['humidity'],
            "pressure": data['main']['pressure']
        }
        return weather
    else:
        return None

weather_function = {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city name",
                    },
                    "temperature": {
                        "type": "number",
                        "description": "The temperature in Celsius",
                    },
                    "humidity": {
                        "type": "number",
                        "description": "The humidity in percentage",
                    },
                    "pressure": {
                        "type": "number",
                        "description": "The pressure",
                    },

                },
                "additionalProperties": False,
                "required": ["location", "temperature", "humidity", "pressure"],
            },
        }
    }

In [ ]:
completion = client.beta.chat.completions.parse(
    model=MODEL,
    messages=[
        {
            "role": "user",
            "content": "what's the weather in Jaffna?",
        },
    ],
    tools=[weather_function],
)

In [ ]:
print(completion.choices[0].message.tool_calls)

[ParsedFunctionToolCall(id='call_zbErlrVvBFjWOzKZOmuxt5uB', function=ParsedFunction(arguments='{"location":"Jaffna","temperature":30,"humidity":70,"pressure":1010}', name='get_weather', parsed_arguments={'location': 'Jaffna', 'temperature': 30, 'humidity': 70, 'pressure': 1010}), type='function')]


In [ ]:
print(completion.choices[0].message.tool_calls[0].function.parsed_arguments)

{'location': 'Jaffna', 'temperature': 30, 'humidity': 70, 'pressure': 1010}
